# Solutions to Practice Problems based on Week 7 content

## General Course Logistics

* Work on your project! This week you should finish your implementation and start working on your solution analysis. Think about whether your solution "makes sense." Do you need to make any changes to your model? Add new assumptions? Remove assumptions? Change your input data? Continue working on the report. Remember, writing as you go will save you a lot of time at the end of the semester!


## Integer Programming Basics


### Logical Modeling practice

Let's suppose the CS524 class has decided collectivley to adopt some puppies. There are 6 different breeds ofdogs we can adopt. Each breed has a minimum and maximum number of puppies if we adopt any of that breed (we have to adopt entire litters so no puppies are left alone!). These restrictions, along with the expected happiness we get from each breed, are in the following table:

|Breed|Min adopted|Max adopted|Happiness|
|:----|:---------|:--------|:------|
|Golden retriever|5|7|8|
|Shiba Inu|2|10|10|
|Great Dane|3|5|4|
|Pomeranian|6|13|5|
|Water Spaniel (WI state dog!)|3|15|9|
|Husky|7|10|6|

We have decided that because of the personality of the Shiba Inu, the total number of Shiba Inu puppies adopted should be no more than the combined number of Golden Retriver, Water Spaniel, and Husky puppies. In addition, if any Pomeranians are adopted, we require that at least the minimum number of Great Dane puppies are adopted. We will adopt 40 total dogs and obviously we want to maximize our total happiness. Which breeds of puppies should we adopt, and how many of each?



In [2]:
dogs = [:golden, :shiba, :greatdane, :pomeranian, :spaniel, :husky]
min_adopt = Dict(zip(dogs,[5 2 3 6 3 7]))
max_adopt = Dict(zip(dogs,[7 10 5 13 15 10]))
happ = Dict(zip(dogs,[8 10 4 5 9 6]))

using JuMP, Gurobi

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m,"OutputFlag",0)

@variable(m, z[dogs], Bin) # binary variable to determine whether we adopt any dog i
@variable(m, x[dogs], Int) # integer variable to determine how many of dog i

@objective(m, Max, sum(happ[i]*x[i] for i in dogs)) # maximize happiness

# adopt at least the minimimum if we adopt any. enforce the logic x > 0 -> x >= min
@constraint(m, min_constr[i in dogs], x[i] >= min_adopt[i]*z[i])
# adopt no more than the maximum if we adopt any. enforce the logic z = 0 -> x = 0
@constraint(m, max_constr[i in dogs], x[i] <= max_adopt[i]*z[i])

# constraint on the total number of Shiba puppies
# we can't adopt any shiba puppies
# unless we adopt at least that many of the other dogs
@constraint(m, x[:shiba] <= x[:golden] + x[:spaniel] + x[:husky])
# an alternative interpretation is that the number of shiba puppies can be anything if 
# no dogs of the other breeds are adopted (only relevant if shiba AND other puppies adopted).
# under this interpretation, we want to enforce the logic that if 
# x[:golden] + x[:spaniel] + x[:husky] > 0, then 
# x[:golden] + x[:spaniel] + x[:husky] >= x[:shiba]
# put a variable in between. x[:golden] + x[:spaniel] + x[:husky] > 0 => delta = 1
# or, the contrapositive: delta = 1 => x[:golden] + x[:spaniel] + x[:husky] = 0
# then delta = 1 => x[:golden] + x[:spaniel] + x[:husky] >= x[:shiba]
# @variable(m, d, Bin)
# # constraint that enforces no golden, spaniel, or huskies adopted if delta = 0
# @constraint(m, x[:golden] + x[:spaniel] + x[:husky] <= d*40)
# # constraint that enforces if delta = 1, then x[:shiba] <= x[:golden] + x[:spaniel] + x[:husky]
# # can be found using Slide of Trix
# @constraint(m, x[:shiba] - (x[:golden] + x[:spaniel] + x[:husky]) + d*10 <= 10)

# cosntraint that z[:pom] = 1 -> x[:greatdane] >= min
@constraint(m, x[:greatdane] >= min_adopt[:greatdane]*z[:pomeranian])
# adopt 40 dogs
@constraint(m, sum(x) == 40)

optimize!(m)

println("Adopt the following puppies: ")
for i in dogs
    if value(z[i]) > 0.001
        println(i, ": ", value(x[i]))
    end
end

Academic license - for non-commercial use only - expires 2022-06-27
Adopt the following puppies: 
golden: 7.0
shiba: 10.0
spaniel: 15.0
husky: 8.0


### Set Covering

Thanos is here! Once again, Earth is relying on the Avengers for protection. For convenience, the United Nations has divided up the world into 8 diﬀerent zones: North America (NA), South America (SA), Eurasia (EU), North Africa (NF), South Africa (SF), East Asia (EA), South/West Asia (WA), and Oceania (OC). The number of hours required to get from each zone to each other zone by Quinjet is given in the following table:

|Zone| NA| SA| EU| NF| SF| EA| WA| OC|
|:--|:--|:--|:--|:--|:--|:--|:--|:--|
| NA|0| 5 |7| 8| 10 |15| 11| 16| 
|SA |5| 0| 11| 10 |9 |15| 12| 13|
|EU| 7| 11 |0 |4| 7| 9| 5| 19 |
|NF |8 |10| 4 |0 |3 |8 |3 |18|
|SF| 10 |9 |7 |3 |0 |10| 4| 16|
|EA |15| 15| 9| 8| 10 |0 |3| 7 |
|WA| 11| 12 |5 |3 |4 |3 |0| 15 |
|OC| 16| 13| 19| 18| 16| 7| 15| 0|

A rough estimate of the population in each district (in millions) as follows: NA: 580, SA: 420, EU: 900, NF: 250, SF: 200, EA: 1700, WA: 400, OC: 40. The UN have declared there are going to be $n$ Avenger locations. The problem is, the UN isn’t sure how many of the Avengers will be available during the defense against Thanos. Their goal is to maximize the number of people that live within 3 hours of an Avenger. We have been asked to assist the UN by determining where to place the Avengers for diﬀerent values of $n$. Solve the problem for $n = 1,2,3,4$.


In [3]:
n = 1

locs = [:NA, :SA, :EU, :NF, :SF, :EA, :WA, :OC]

using NamedArrays

trav_time = NamedArray([0 5 7 8 10 15 11 16
        5 0 11 10 9 15 12 13
        7 11 0 4 7 9 5 19
        8 10 4 0 3 8 3 18
        10 9 7 3 0 10 4 16
        15 15 9 8 10 0 3 7
        11 12 5 3 4 3 0 15
        16 13 19 18 16 7 15 0], (locs, locs), ("locs", "locs"))

pop = Dict(zip(locs,[580 420 900 250 200 1700 400 40]))

cover = NamedArray(zeros(length(locs),length(locs)), (locs, locs), ("locs", "locs"))
for i in locs
    for j in locs
        if trav_time[i,j] <= 3
            cover[i,j] = 1
        end
    end
end

using JuMP, Gurobi

# for each possible number of available Avengers
for n in [1,2,3,4]
    m = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m,"OutputFlag",0)

    # binary variable representing whether we put an Avenger in location i \in locs
    @variable(m, x[locs], Bin)
    # binary variable representing whether location i in locs is covered
    @variable(m, c[locs], Bin)

    # maximize the population covered
    @objective(m, Max, sum(pop[j]*c[j] for j in locs))

    # constraint that the number of locations we place Avengers = n
    @constraint(m, sum(x) == n)
    
    # cover constraint. If loc j is covered by loc i, this will be 1*x[i] >= c[j]
    # this also enforces logic that if we want c[j] = 1, at least one x[i] that covers loc j must be 1
    @constraint(m, cov_con[j in locs], sum(cover[i,j]*x[i] for i in locs) >= c[j])
    
    optimize!(m)
    
    for i in locs
        if value(x[i]) == 1
            println("Place Avegner in district ", i)
            println("The following districts will be covered: ")
            for j in locs
                if value(c[j]) == 1
                    println(j)
                end
            end
        end
    end
end

Academic license - for non-commercial use only - expires 2022-06-27
Place Avegner in district WA
The following districts will be covered: 
NF
EA
WA
Academic license - for non-commercial use only - expires 2022-06-27
Place Avegner in district EU
The following districts will be covered: 
EU
NF
EA
WA
Place Avegner in district WA
The following districts will be covered: 
EU
NF
EA
WA
Academic license - for non-commercial use only - expires 2022-06-27
Place Avegner in district NA
The following districts will be covered: 
NA
EU
NF
EA
WA
Place Avegner in district EU
The following districts will be covered: 
NA
EU
NF
EA
WA
Place Avegner in district WA
The following districts will be covered: 
NA
EU
NF
EA
WA
Academic license - for non-commercial use only - expires 2022-06-27
Place Avegner in district NA
The following districts will be covered: 
NA
SA
EU
NF
EA
WA
Place Avegner in district SA
The following districts will be covered: 
NA
SA
EU
NF
EA
WA
Place Avegner in district EU
The following dis